# Loading and plotting a photon spectrum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from agepy.spec.photons import Spectrum, DldAnodeUVW

In [ ]:
anode = DldAnodeUVW(-98.8)

In [ ]:
spectrum = Spectrum.from_h5(
    "spectrum.h5",
    raw = "dld_rd#raw",
    time = 600,
    anode = anode,
    target_density = "baratron#value",
    intensity_upstream = "mirror#value",
)

In [ ]:
# Define a ROI
roi = {
    "x": {"min": 0.16, "max": 0.80},
    "y": {"min": 0.42, "max": 0.52},
}

In [ ]:
# Simulate detector efficiencies
qeff_xe = np.histogram([], bins=512, range=(0, 1))[1]
qeff_val = np.random.normal(0.9, 0.1, 512)
qeff_err = np.ones(512) * 0.1

In [ ]:
# Define wavelength calibration
a0 = (128.67, 0.07)
a1 = (38.94, 0.01)
calib = (a0, a1)

In [ ]:
# Define a binning for the spectrum
xedges = np.histogram([], bins=512, range=(a0[0], a0[0] + a1[0]))[1]
xvalues = (xedges[:-1] + xedges[1:]) * 0.5

In [ ]:
spec, errors = spectrum.spectrum(
    xedges,
    roi = roi,
    qeff = (qeff_val, qeff_err, qeff_xe),
    calib = calib,
    uncertainties = "jacobi",
)

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(xvalues, spec[1], yerr=errors, fmt="o")